# Session 3: 상관관계와 예측 모델 (Integrated)

## 🎯 학습 목표
- **Deep Theory**: 공분산과 상관계수, 그리고 최소제곱법(OLS)의 미분 과정을 이해합니다.
- **Practice**: 경사하강법(Gradient Descent)을 직접 구현하여 선형 회귀 모델을 학습시킵니다.
- **Visualization**: 산점도와 회귀선을 통해 데이터의 패턴과 이상치를 눈으로 확인합니다. (Anscombe's Quartet)
- **Concepts**: 상관관계와 인과관계를 구분하고, 선형 모델의 한계를 배웁니다.
- **Experiment**: 노이즈와 이상치가 모델 학습에 미치는 영향을 실험합니다.

---

# Part 1. 📐 Deep Theory: 관계와 학습

## 1. 공분산(Covariance)과 상관계수(Correlation)

두 변수 $X, Y$가 함께 변하는 정도를 측정합니다.

$$ Cov(X, Y) = \frac{1}{n-1} \sum (x_i - \bar{x})(y_i - \bar{y}) $$

- **공분산의 문제**: $X, Y$의 단위(Scale)에 따라 값이 너무 커지거나 작아집니다.
- **해결 (정규화)**: 각 변수의 표준편차로 나누어 $-1 \sim 1$ 사이 값으로 맞춥니다. -> **상관계수($r$)**

$$ r_{xy} = \frac{Cov(X, Y)}{s_x s_y} $$

## 2. 최소제곱법 (Ordinary Least Squares)

데이터 $y$와 예측 $\hat{y} = wx + b$ 사이의 오차 제곱합(SSE)을 최소화하는 $w, b$를 찾습니다.

$$ Loss = \sum (y_i - (wx_i + b))^2 $$

이 Loss 함수는 아래로 볼록한(Convex) 2차 함수이므로, **미분해서 0이 되는 지점**이 최솟값입니다.

$$ \frac{\partial Loss}{\partial w} = -2 \sum x_i(y_i - (wx_i + b)) = 0 $$
$$ \frac{\partial Loss}{\partial b} = -2 \sum (y_i - (wx_i + b)) = 0 $$

---
# Part 2. 💻 Manual Implementation: 경사하강법 (Gradient Descent)

미분 공식을 이용하여, 조금씩 경사를 타고 내려가 최적의 $w, b$를 찾는 알고리즘을 구현합니다.
$$ w_{new} = w_{old} - \alpha \frac{\partial Loss}{\partial w} $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 가상의 데이터 생성 (y = 2x + 1 + noise)
np.random.seed(42)
X = np.random.rand(100, 1)
y = 2 * X + 1 + np.random.randn(100, 1) * 0.1

def gradient_descent(X, y, lr=0.1, epochs=1000):
    w = np.random.randn(1, 1)
    b = np.random.randn(1, 1)
    N = len(y)
    
    for _ in range(epochs):
        # 예측
        y_pred = X * w + b
        
        # 편미분 (Gradient 계산)
        # dLoss/dw = -2/N * sum(x * error)
        error = y - y_pred
        dw = -2 * np.sum(X * error) / N  # 평균제곱오차 기준
        db = -2 * np.sum(error) / N
        
        # 파라미터 업데이트
        w = w - lr * dw
        b = b - lr * db
        
    return w, b

w_hat, b_hat = gradient_descent(X, y)
print(f"Estimated: w={w_hat[0][0]:.4f}, b={b_hat[0][0]:.4f} (Actual: w=2, b=1)")

---
# 🚀 Practical Usage: Scikit-learn & Pandas

손으로 짜는 경사하강법은 오래 걸립니다. `sklearn`은 최적화된 C++ 백엔드로 순식간에 답을 구해줍니다.

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd

# 1. Linear Regression (OLS)
model = LinearRegression()
model.fit(X, y)

print(f"Sklearn Coef (w): {model.coef_[0][0]:.4f}")
print(f"Sklearn Intercept (b): {model.intercept_[0]:.4f}")

# 2. Correlation Matrix
df = pd.DataFrame({'X': X.flatten(), 'Y': y.flatten()})
print("\nCorrelation Table:")
print(df.corr())

---
# Part 3. 📖 Concepts & Visualization: 데이터의 진실을 보는 눈

## 1. 왜 시각화가 중요한가? (Anscombe's Quartet)

유명한 예시인 '앤스컴의 콰르텟'은 4개의 데이터셋이 **평균, 분산, 상관계수, 회귀선까지 모두 동일**합니다.
하지만 그래프를 그려보면 완전히 다른 데이터임을 알 수 있습니다.

> **Lesson**: "숫자(요약 통계량)만 믿지 말고, 반드시 그려보세요."

## 2. Seaborn `regplot`: 한 줄로 끝내는 시각화
산점도(Scatter)와 회귀선(Line), 그리고 95% 신뢰구간(Confidence Interval)을 한 번에 그려줍니다.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 예제 데이터: 팁 데이터셋 (식사 금액 vs 팁)
tips = sns.load_dataset("tips")

plt.figure(figsize=(10, 5))

# total_bill(X)이 커지면 tip(Y)도 커지는가?
sns.regplot(x="total_bill", y="tip", data=tips, line_kws={"color": "red"})
plt.title("Total Bill vs Tip (with Regression Line)")
plt.show()

### 잔차 (Residual): 데이터와 모델의 거리
위 그래프에서 빨간 선(예측)과 파란 점(실제) 사이의 거리가 바로 **잔차(Error)**입니다.
- 점들이 선에 딱 붙어있다 $\rightarrow$ 모델 성능이 좋다 ($R^2$ 높음)
- 점들이 중구난방이다 $\rightarrow$ 예측이 어렵다

## 3. Correlation Analysis Deep Dive (상관분석 심화)

### A. 공분산(Covariance)의 한계
"키($X$)와 몸무게($Y$)의 공분산이 500이다"라는 말은 아무 의미가 없습니다.
- 키 단위를 `cm`에서 `m`로 바꾸면 공분산은 5로 줄어듭니다.
- **단위(Scale)**에 따라 값이 널뛰기하므로 절대적인 비교가 불가능합니다.
- 그래서 단위를 없앤(정규화한) **상관계수($r$)**를 사용합니다.

### B. 상관계수($r$) 해석 가이드
- **값의 범위**: $-1 \sim 1$
- **강도(Strength)**:
    - $|r| > 0.7$: 강한 상관관계
    - $|r| < 0.3$: 약한 상관관계 (거의 관계 없음)
- **방향(Direction)**:
    - 양수(+): 비례 (공부 시간 $\uparrow$ $\rightarrow$ 성적 $\uparrow$)
    - 음수(-): 반비례 (결석 횟수 $\uparrow$ $\rightarrow$ 성적 $\downarrow$)
- **⚠️ 함정 (Causation)**: "아이스크림 판매량과 익사 사고율은 상관계수가 높다." $\rightarrow$ 아이스크림이 익사의 원인인가? 아님. **'여름(기온)'**이라는 제3의 변수 때문.

### C. Heatmap 제대로 읽기
- **대칭성**: 대각선(/)을 기준으로 대칭(Decalcomanie)입니다.
- **대각선=1**: 자기 자신과의 상관계수는 무조건 1입니다.
- **다중공선성 경고**: 입력 변수($X$)끼리 너무 붉은색(높은 상관관계)이면 좋지 않습니다. (모델이 헷갈려 함)

In [ ]:
plt.figure(figsize=(8, 6))
# 팁 데이터셋의 숫자형 변수들만 상관관계 분석
corr_matrix = tips.corr(numeric_only=True)

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlation Heatmap: Tips Dataset")
plt.show()

---
# Part 4. 🧪 Experiment: 이상치(Outlier)의 영향력

**"이상치 하나가 전체 모델을 망가뜨릴 수 있다"**는 것을 눈으로 확인해 봅시다.

In [ ]:
# 정상 데이터 (우상향 패턴)
X_normal = np.array([1, 2, 3, 4, 5])
y_normal = np.array([1, 2, 3, 4, 5])

# 이상치 추가 (X=10인데 y=-100인 말도 안 되는 데이터)
X_outlier = np.append(X_normal, 10)
y_outlier = np.append(y_normal, -100)

# 시각화
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# 정상 데이터
sns.regplot(x=X_normal, y=y_normal, ax=ax[0], color='blue')
ax[0].set_title("Normal Data (Perfect Fit)")

# 이상치 포함 데이터
sns.regplot(x=X_outlier, y=y_outlier, ax=ax[1], color='red')
ax[1].set_title("With One Outlier (Model Broken)")

plt.show()

> **Discussion**: 단 하나의 이상치가 회귀선(빨간 선)을 완전히 엉뚱한 방향으로 끌고 내려갑니다. 이것이 **전처리(이상치 제거)**가 중요한 이유입니다.

---
# Part 5. 🛠 Library Mapping: 실전 최적화

우리는 반복문으로 경사하강법을 직접 짰지만(`Part 2`), 실무에서는 `SGDRegressor`를 씁니다.
반면 `LinearRegression`은 경사하강법이 아니라 행렬 연산(OLS)을 사용해 정답을 한 방에 찾습니다.

### 1. SGDRegressor (경사하강법)
- 데이터 하나씩 보면서 조금씩 w, b를 수정 (Deep Learning Style)
- 데이터가 엄청 많을 때 빠름

### 2. LinearRegression (OLS)
- 수식으로 한 방에 푼다 (Exact Solution)
- 데이터가 적당할 때 가장 정확함

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

# SGD는 데이터 스케일에 민감하므로 정규화 필수!
X_scaled = StandardScaler().fit_transform(X)
y_flatten = y.flatten()

sgd = SGDRegressor(max_iter=1000, tol=1e-3, learning_rate='constant', eta0=0.01)
sgd.fit(X_scaled, y_flatten)

print(f"SGD Coef (w): {sgd.coef_[0]:.4f}")
print(f"SGD Intercept (b): {sgd.intercept_[0]:.4f}")
# LinearRegression과 거의 비슷한 결과가 나오지만, 과정은 다릅니다.